In [0]:
from pyspark.sql.functions import datediff, current_date, avg
from pyspark.sql.types import IntegerType

#### Question 1: What was the average time each driver spent at the pit stop for each race?

In [0]:
df_pitstops = spark.read.csv('s3://columbia-gr5069-main/raw/pit_stops.csv', header=True)

In [0]:
display(df_pitstops)

In [0]:
avg_pitstop_time = df_pitstops.groupBy("raceId", "driverId").agg(avg("duration").alias("avg_pitstop_time"))

display(avg_pitstop_time)